In [1]:
# import the necessary packages
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from imutils.video import VideoStream
from imutils.video import FPS
import time



In [2]:

# Set the directory you want to start from
rootDir = '.'
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Found directory: %s' % dirName)
    for fname in fileList:
        print('\t%s' % fname)

Found directory: .
	Face_detection_Webcam.ipynb
	openface.nn4.small2.v1.t7
	Face_recognition_Webcam.ipynb
Found directory: ./dataset
Found directory: ./dataset/claudia
	clau_6.jpg
	clau_5.jpg
	clau_1.jpeg
	clau_4.jpg
	clau_2.jpg
	clau_3.jpg
Found directory: ./dataset/unknown
	desconocido_2.jpeg
	desconocido_1.jpeg
Found directory: ./dataset/daniel
	mi_foto.jpg
	daniel_2.jpg
	daniel_1.jpg
	daniel_3.jpg
	daniel_4.jpg
Found directory: ./.ipynb_checkpoints
	Face_recognition_Webcam-checkpoint.ipynb
	Face_detection_Webcam-checkpoint.ipynb
Found directory: ./output
	embeddings.pickle
	le.pickle
	recognizer.pickle
Found directory: ./face_detection_model
	res10_300x300_ssd_iter_140000.caffemodel
	deploy.prototxt.txt


# 0.- Load face detector Model


In [3]:
# load our serialized face detector from disk
print("[INFO] loading face detector...")
path_prototxt = "face_detection_model/deploy.prototxt.txt"
path_model = "face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"
th_confidence = 0.5
detector = cv2.dnn.readNetFromCaffe(path_prototxt, path_model)
# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedding_model = "openface.nn4.small2.v1.t7"
embeddings_output = "output/embeddings.pickle"
embedder = cv2.dnn.readNetFromTorch(embedding_model)


[INFO] loading face detector...
[INFO] loading face recognizer...


# 1.-Extract embeddings from face dataset

In [4]:
# grab the paths to the input images in our dataset
print("[INFO] quantifying faces...")
path_dataset = "dataset"
imagePaths = list(paths.list_images(path_dataset))
# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []
# initialize the total number of faces processed
total = 0

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    print("[INFO] processing image {}/{}".format(i + 1,
        len(imagePaths)))
    name = imagePath.split(os.path.sep)[-2]
    # load the image, resize it to have a width of 600 pixels (while
    # maintaining the aspect ratio), and then grab the image
    # dimensions
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)
    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    # ensure at least one face was found
    if len(detections) > 0:
        # we're making the assumption that each image has only ONE
        # face, so find the bounding box with the largest probability
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]
        # ensure that the detection with the largest probability also
        # means our minimum probability test (thus helping filter out
        # weak detections)
        if confidence > th_confidence:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # extract the face ROI and grab the ROI dimensions
            face = image[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]
            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue
                
            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()
            # add the name of the person + corresponding face
            # embedding to their respective lists
            knownNames.append(name)
            knownEmbeddings.append(vec.flatten())
            total += 1
            
# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open(embeddings_output, "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] quantifying faces...
[INFO] processing image 1/13
[INFO] processing image 2/13
[INFO] processing image 3/13
[INFO] processing image 4/13
[INFO] processing image 5/13
[INFO] processing image 6/13
[INFO] processing image 7/13
[INFO] processing image 8/13
[INFO] processing image 9/13
[INFO] processing image 10/13
[INFO] processing image 11/13
[INFO] processing image 12/13
[INFO] processing image 13/13
[INFO] serializing 12 encodings...


# 2.- Train recognition model

In [5]:
output_recognizer = "output/recognizer.pickle"
output_label = "output/le.pickle"
embeddings_output = "output/embeddings.pickle"

# load the face embeddings
print("[INFO] loading face embeddings...")
data = pickle.loads(open(embeddings_output, "rb").read())
# encode the labels
print("[INFO] encoding labels...")
le = LabelEncoder()
labels = le.fit_transform(data["names"])

# train the model used to accept the 128-d embeddings of the face and
# then produce the actual face recognition
print("[INFO] training model...")
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(data["embeddings"], labels)
# write the actual face recognition model to disk
f = open(output_recognizer, "wb")
f.write(pickle.dumps(recognizer))
f.close()
# write the label encoder to disk
f = open(output_label, "wb")
f.write(pickle.dumps(le))
f.close()

[INFO] loading face embeddings...
[INFO] encoding labels...
[INFO] training model...


# 3.- Load Recognition Model

In [6]:
path_prototxt = "face_detection_model/deploy.prototxt.txt"
path_model = "face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"
th_confidence = 0.5
output_recognizer = "output/recognizer.pickle"
output_label = "output/le.pickle"
embeddings_output = "output/embeddings.pickle"
embedding_model = "openface.nn4.small2.v1.t7"

# load our serialized face detector from disk
print("[INFO] loading face detector...")

detector = cv2.dnn.readNetFromCaffe(path_prototxt, path_model)
# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(embedding_model)
# load the actual face recognition model along with the label encoder
recognizer = pickle.loads(open(output_recognizer, "rb").read())
le = pickle.loads(open(output_label, "rb").read())


[INFO] loading face detector...
[INFO] loading face recognizer...


# 4.- Recognition in Webcam

In [7]:
# initialize the video stream, then allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)
# start the FPS throughput estimator
fps = FPS().start()
# loop over frames from the video file stream
while True:
    # grab the frame from the threaded video stream
    frame = vs.read()
    # resize the frame to have a width of 600 pixels (while
    # maintaining the aspect ratio), and then grab the image
    # dimensions
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]
    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)
    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections
        if confidence > th_confidence:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # extract the face ROI
            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]
            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue
        # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()
            # perform classification to recognize the face
            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]
            # draw the bounding box of the face along with the
            # associated probability
            text = "{}: {:.2f}%".format(name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                (0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    # update the FPS counter
    fps.update()
                
            # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))
# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] starting video stream...
[INFO] elasped time: 126.51
[INFO] approx. FPS: 17.10


In [8]:
vs.stop()